In [1]:
# DATA SETUP

In [17]:
import numpy as np
import pandas as pd

from qbayes_tools import *

In [18]:
#################################################
# COVID-19 EXAMPLES:

In [24]:
def get_lesser_model_nodes(filename='data/lesser_model_data.csv'):
    lesserdata = pd.read_csv(filename)
    
    statedataStayHome = {'MarHome' : lesserdata['MarHome'], 'AprHome' : lesserdata['AprHome'], 'MayHome' : lesserdata['MayHome'], 'JunHome' : lesserdata['JunHome']}
    StayHome = Node("StayHome", np.ndarray.flatten(np.array(pd.DataFrame(data=statedataStayHome))), states = {"No" : 0, "Yes" : 1})

    statedataTests = {'MarTest' : lesserdata['MarTest'], 'AprTest' : lesserdata['AprTest'], 'MayTest' : lesserdata['MayTest'], 'JunTest' : lesserdata['JunTest']}
    Tests = Node("Tests", np.ndarray.flatten(np.array(pd.DataFrame(data=statedataTests))), states = {"GT5" : 0, "LE5" : 1})

    statedataCases = {'MarCases' : lesserdata['MarCases'], 'AprCases' : lesserdata['AprCases'], 'MayCases' : lesserdata['MayCases'], 'JunCases' : lesserdata['JunCases']}
    Cases = Node("Cases", np.ndarray.flatten(np.array(pd.DataFrame(data=statedataCases))), states = {"Inc" : 0, "noInc" : 1}, parents = ["Tests", "StayHome"])
                 
    return Cases, Tests, StayHome

In [25]:
def get_mallard_model_states(filename='data/mallardmodeldata.csv'):
    mallarddata = pd.read_csv(filename)
    Cases = Node("Cases", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarCases':mallarddata['MarCases'], 'AprCases':mallarddata['AprCases'], 'MayCases':mallarddata['MayCases'], 'JunCases':mallarddata['JunCases']}))),
                 states = {"Inc" : 0, "Min" : 1, "Mod" : 2, "Maj" : 3}, parents = ["Test", "Mask", "Work", "Rec"])
    Test = Node("Test", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarTest':mallarddata['MarTest'],'AprTest':mallarddata['AprTest'],'MayTest':mallarddata['MayTest'], 'JuneTest':mallarddata['JunTest']}))),
                states = {"GT5" : 0, "LE5" : 1})
    Mask = Node("Mask", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarMask':mallarddata['MarMask'],'AprMask':mallarddata['AprMask'],'MayMask':mallarddata['MayMask'],'JunMask':mallarddata['JunMask']}))),
                states = {"No" : 0, "Yes" : 1})
    Work = Node("Work", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarWork':mallarddata['MarWork'],'AprWork':mallarddata['AprWork'],'MayWork':mallarddata['MayWork'],'JunWork':mallarddata['JunWork']}))),
                states = {"Inc" : 0, "Min" : 1, "Mod" : 2, "Maj" : 3})
    Rec = Node("Rec", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarRec':mallarddata['MarRec'],'AprRec':mallarddata['AprRec'],'MayRec':mallarddata['MayRec'],'JunRec':mallarddata['JunRec']}))),
               states = {"Inc" : 0, "Min" : 1, "Mod" : 2, "Maj" : 3})
    Death = Node("Death", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarDeath':mallarddata['MarDeath'],'AprDeath':mallarddata['AprDeath'],'MayDeath':mallarddata['MayDeath'],'JunDeath':mallarddata['JunDeath']}))),
                 states = {"Inc" : 0, "notInc" : 1}, parents = ["Cases", "Age"])
    Age = Node("Age", np.ndarray.flatten(np.array(pd.DataFrame(data={'MarAge':mallarddata['MarAge'],'AprAge':mallarddata['AprAge'],'MayAge':mallarddata['MayAge'],'JunAge':mallarddata['JunAge']}))),
               states = {"Old" : 0, "Young" : 1})
    return Cases, Test, Mask, Work, Rec, Death, Age

In [14]:
#THIS STILL NEEDS TO BE IMPLEMENTED AS MAKING NODE OBJECTS. I STOPPED AFTER UPDATING MALLARD IN THIS WAY

def get_alabio_model_states(filename='data/alabiomodeldata.csv'):
    alabiodata = pd.read_csv(filename)
    Cases = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarCases':alabiodata['MarCases'], 'AprCases':alabiodata['AprCases'], 'MayCases':alabiodata['MayCases'], 'JunCases':alabiodata['JunCases']})))
    Test = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarTest':alabiodata['MarTest'],'AprTest':alabiodata['AprTest'],'MayTest':alabiodata['MayTest'], 'JuneTest':alabiodata['JunTest']})))
    Mask = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarMask':alabiodata['MarMask'],'AprMask':alabiodata['AprMask'],'MayMask':alabiodata['MayMask'],'JunMask':alabiodata['JunMask']})))
    Work = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarWork':alabiodata['MarWork'],'AprWork':alabiodata['AprWork'],'MayWork':alabiodata['MayWork'],'JunWork':alabiodata['JunWork']})))
    Rec = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarRec':alabiodata['MarRec'],'AprRec':alabiodata['AprRec'],'MayRec':alabiodata['MayRec'],'JunRec':alabiodata['JunRec']})))
    Death = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarDeath':alabiodata['MarDeath'],'AprDeath':alabiodata['AprDeath'],'MayDeath':alabiodata['MayDeath'],'JunDeath':mallarddata['JunDeath']})))
    Age = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarAge':alabiodata['MarAge'],'AprAge':alabiodata['AprAge'],'MayAge':alabiodata['MayAge'],'JunAge':alabiodata['JunAge']})))
    Race = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarRace':alabiodata['MarRace'],'AprRace':alabiodata['AprRace'],'MayRace':alabiodata['MayRace'],'JunRace':alabiodata['JunRace']})))
    Poverty = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarPoverty':alabiodata['MarPoverty'],'AprPoverty':alabiodata['AprPoverty'],'MayPoverty':alabiodata['MayPoverty'],'JunPoverty':alabiodata['JunPoverty']})))
    Health = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarHealth':alabiodata['MarHealth'],'AprHealth':alabiodata['AprHealth'],'MayHealth':alabiodata['MayHealth'],'JunHealth':alabiodata['JunHealth']})))
    return Cases, Test, Mask, Work, Rec, Death, Age, Race, Poverty, Health
    

In [15]:
#################################################
"""
# MAKE YOUR OWN HERE!
def MyDataSetup(*input):

##########################
# INPUT #
# input    str    filename with data
# **kwargs  whatever else you need

# OUTPUT #
# *data     tuple of Nodes
    return data


"""

'\n# MAKE YOUR OWN HERE!\ndef MyDataSetup(*input):\n\n##########################\n# INPUT #\n# input    type    whatever you need\n\n# OUTPUT #\n# *data     array(s)    1d numpy array(s) of integer entries where the indices match between datasets\n    return data\n\n\n'

In [20]:
get_lesser_model_nodes()

(<qbayes_tools.Node at 0x2588e538dc8>,
 <qbayes_tools.Node at 0x2588e558d08>)